In [1]:
#1,2. 데이터 가공
import numpy as np
import pandas as pd

#2.
import os

import matplotlib.pyplot as plt

import cv2 as cv

from sklearn.model_selection import train_test_split 

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import VGG16

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization

from tensorflow.keras.models import save_model, load_model

In [2]:
#1. 일별 초당 평균발전량 (date and time&AVG power)데이터 가공

#1-1. csv파일 읽기
power_10_df = pd.read_csv("./Low-res_sky_images_kaggle/Power_measurements/2019_09_10.csv")
power_11_df = pd.read_csv("./Low-res_sky_images_kaggle/Power_measurements/2019_09_11.csv")
power_12_df = pd.read_csv("./Low-res_sky_images_kaggle/Power_measurements/2019_09_12.csv")

#1-2. 날짜&시간을 알려주는 컬럼['date and time']의 자료형태를 시간으로 바꾸기
power_10_df['Date and Time'] = pd.to_datetime(power_10_df['Date and Time'])
power_11_df['Date and Time'] = pd.to_datetime(power_11_df['Date and Time'])
power_12_df['Date and Time'] = pd.to_datetime(power_12_df['Date and Time'])

#1-3. power_10_df, power_11_df, power_12_df을 합치기
power_df = pd.concat([power_10_df, power_11_df, power_12_df], axis = 0)
power_df  #8640 rows × 2 columns

#1-4. 결측값 확인하기_1차
power_df.isnull().sum()

#1-5.결측값 제거하기
power_df.dropna(inplace=True)

#1-6. 결측값 확인하기_2차
power_df.isnull().sum()

power_df  #값은 (8175, 2)

#1-7. 겹치는 시간대에 대해서 통합하기
power_df_avg = power_df.groupby('Date and Time').mean()
power_df_avg               #4088 rows × 1 columns

power_df_avg['DateTime'] = power_df_avg.index
power_df_avg = power_df_avg.reset_index(drop=True)
power_df_avg            #4088 rows × 2 columns

,AVG power,DateTime
0,0.0,2019-09-10 00:00:00
1,0.0,2019-09-10 00:01:00
2,0.0,2019-09-10 00:02:00
3,0.0,2019-09-10 00:03:00
4,0.0,2019-09-10 00:04:00
...,...,...
4083,0.0,2019-09-12 20:03:00
4084,0.0,2019-09-12 20:04:00
4085,0.0,2019-09-12 20:05:00
4086,0.0,2019-09-12 20:06:00


In [3]:
#2. (사진번호, 시, 분, 초 )데이터의 가공

#2-1. csv파일 읽기
time_10_df = pd.read_csv("./Low-res_sky_images_kaggle/Sky_images/10_09_2019/time.csv")
time_11_df = pd.read_csv("./Low-res_sky_images_kaggle/Sky_images/11_09_2019/time.csv")
time_12_df = pd.read_csv("./Low-res_sky_images_kaggle/Sky_images/11_09_2019/time.csv")

#2-1. csv파일 읽기
time_10_df = time_10_df[:-1]
time_11_df = time_11_df[:-1]
time_12_df = time_11_df

# time_10_df  #2999 rows × 4 columns
# time_11_df  #2999 rows × 4 columns
# time_12_df  #2999 rows × 4 columns


#2-2. 날짜&시간을 알려주는 컬럼['date and time']의 생성
time_10_df['Date and Time'] = pd.to_datetime(time_10_df[['Hours', 'Minutes', 'Seconds']].assign(Day=10, Month=9, Year=2019))
time_10_df = time_10_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)
time_11_df['Date and Time'] = pd.to_datetime(time_11_df[['Hours', 'Minutes', 'Seconds']].assign(Day=11, Month=9, Year=2019))
time_11_df = time_11_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)
time_12_df['Date and Time'] = pd.to_datetime(time_12_df[['Hours', 'Minutes', 'Seconds']].assign(Day=12, Month=9, Year=2019))
time_12_df = time_12_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)


#2-3. 사진파일의 경로를 알려주는 컬럼['FilePath']의 생성
folder = './Low-res_sky_images_kaggle/Sky_images/10_09_2019/Images/'
time_10_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_10_df['Image Number']]
folder = './Low-res_sky_images_kaggle/Sky_images/11_09_2019/Images/'
time_11_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_11_df['Image Number']]
folder = './Low-res_sky_images_kaggle/Sky_images/12_09_2019/Images/'
time_12_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_12_df['Image Number']]

# time_10_df  #2999 rows × 4 columns
# time_11_df  #2999 rows × 3 columns
# time_12_df  #2999 rows × 3 columns


#2-4. 결측값 제거하기
time_10_df.dropna(inplace=True)
time_11_df.dropna(inplace=True)
time_12_df.dropna(inplace=True)

# time_10_df  #2999 rows × 3 columns
# time_11_df  #2999 rows × 3 columns
# time_12_df  #2999 rows × 3 columns

In [4]:
#3.분단위로 데이터 정리
#3-1. 컬럼[Date and Time]에서 같은 분에 해당하는 데이터를 모으기 위해서 초가 없어진 컬럼[DateTime]을 생성
time_10_df['DateTime'] = time_10_df['Date and Time'].dt.floor('min')
time_11_df['DateTime'] = time_11_df['Date and Time'].dt.floor('min')
time_12_df['DateTime'] = time_12_df['Date and Time'].dt.floor('min')


#3-2. 분단위로 시간 데이터가 정리된 컬럼[DateTime]을 같은 분단위 시간끼리 묶기
grouped = time_10_df.groupby('DateTime')
grouped_11 = time_11_df.groupby('DateTime')
grouped_12 = time_12_df.groupby('DateTime')

#3-3.
aggregated = grouped['FilePath'].agg(lambda x: ','.join(x))
aggregated_11 = grouped_11['FilePath'].agg(lambda x: ','.join(x))
aggregated_12 = grouped_12['FilePath'].agg(lambda x: ','.join(x))

#3-4.
result = aggregated.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_11 = aggregated_11.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_12 = aggregated_12.reset_index().rename(columns={'FilePath': 'FilePaths'})

#3-5.
result['NumberFiles'] = result['FilePaths'].str.split(',').str.len()
result_11['NumberFiles'] = result_11['FilePaths'].str.split(',').str.len()
result_12['NumberFiles'] = result_12['FilePaths'].str.split(',').str.len()


#3-6.
images_df = pd.concat([result, result_11, result_12], axis = 0)
images_df  #1503 rows × 3 columns

,DateTime,FilePaths,NumberFiles
0,2019-09-10 08:00:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,5
1,2019-09-10 08:01:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
2,2019-09-10 08:02:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
3,2019-09-10 08:03:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
4,2019-09-10 08:04:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
...,...,...,...
496,2019-09-12 16:16:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
497,2019-09-12 16:17:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
498,2019-09-12 16:18:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
499,2019-09-12 16:19:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6


In [5]:
#4-1.
merged_df = pd.merge(power_df_avg, images_df, on='DateTime')
merged_df

,AVG power,DateTime,FilePaths,NumberFiles
0,1350.0,2019-09-10 08:00:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,5
1,1315.0,2019-09-10 08:01:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
2,1340.0,2019-09-10 08:02:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
3,1365.0,2019-09-10 08:03:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
4,1395.0,2019-09-10 08:04:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6
...,...,...,...,...
1498,486.5,2019-09-12 16:16:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
1499,471.5,2019-09-12 16:17:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
1500,474.5,2019-09-12 16:18:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6
1501,476.0,2019-09-12 16:19:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6


In [6]:
bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000]
labels = ['0', '1', '2', '3', '4', '5', '6']
merged_df['category'] = pd.cut(merged_df['AVG power'], bins=bins, labels=labels)
merged_df

,AVG power,DateTime,FilePaths,NumberFiles,category
0,1350.0,2019-09-10 08:00:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,5,1
1,1315.0,2019-09-10 08:01:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6,1
2,1340.0,2019-09-10 08:02:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6,1
3,1365.0,2019-09-10 08:03:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6,1
4,1395.0,2019-09-10 08:04:00,./Low-res_sky_images_kaggle/Sky_images/10_09_2...,6,1
...,...,...,...,...,...
1498,486.5,2019-09-12 16:16:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6,0
1499,471.5,2019-09-12 16:17:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6,0
1500,474.5,2019-09-12 16:18:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6,0
1501,476.0,2019-09-12 16:19:00,./Low-res_sky_images_kaggle/Sky_images/12_09_2...,6,0


In [7]:
#
file_list= []
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        file=merged_df['FilePaths'][i].split(',')[j]
        file_list.append(file)
len(file_list)

8997

In [8]:
y=[]
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        y_=merged_df['category'][i]
        y.append(y_)
len(y)  

8997

In [9]:
# dt=plt.imread(file_list[0])
# dt1=cv.resize(dt,(150,150))
# print(dt1.shape)
# plt.imshow(dt1)

In [10]:
X=np.zeros((8997,150,150,3))

In [11]:
for idx,file in enumerate(file_list):
    dt=plt.imread(file)
    dt=cv.resize(dt,(150,150)).reshape(1,150,150,-1)
    X[idx] = dt[:,:,:,:3]

In [12]:
X_, X_test, y_, y_test=train_test_split(X,y,test_size=.2)
X_train, X_validation, y_train, y_validation=train_test_split(X_,y_)

In [13]:
y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)
y_test = to_categorical(y_test)

In [14]:
X_train[0],y_train[0]

(array([[[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [0., 2., 0.],
         [0., 2., 0.],
         [0., 1., 0.]],
 
        [[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [1., 2., 1.],
         [0., 1., 0.],
         [0., 1., 0.]],
 
        [[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [0., 2., 0.],
         [0., 1., 0.],
         [0., 1., 0.]],
 
        ...,
 
        [[2., 4., 3.],
         [0., 2., 1.],
         [1., 4., 0.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]],
 
        [[0., 4., 3.],
         [2., 4., 3.],
         [0., 4., 3.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]],
 
        [[1., 2., 4.],
         [1., 3., 2.],
         [0., 4., 0.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]]),
 array([0., 1., 0., 0., 0., 0., 0.], dtype=float32))

In [15]:
conv_base=VGG16(include_top=False, input_shape=(150,150,3))

In [16]:
X_train_trans=conv_base.predict(X_train)
X_validation_trans=conv_base.predict(X_validation)

57/57 [==============================] - 42s 733ms/step


In [17]:
X_train_trans.shape

(5397, 4, 4, 512)

In [18]:
#conv_base의 출력값은 4,4,512임
model=Sequential(
    [
        Flatten(input_shape=X_train_trans.shape[1:]),
        Dense(256,activation='relu'),
        BatchNormalization(),
        Dense(128,activation='relu'),
        BatchNormalization(),
        Dense(7,activation='softmax')
    ]
)

In [19]:
optimizer='adam'
loss='categorical_crossentropy'
metrics='accuracy'
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [20]:
model.fit(X_train_trans,y_train,batch_size=32,validation_data=(X_validation_trans,y_validation),epochs=200)

Epoch 1/200


169/169 [==============================] - 6s 21ms/step - loss: 0.4792 - accuracy: 0.8251 - val_loss: 0.4068 - val_accuracy: 0.8594
Epoch 2/200
169/169 [==============================] - 3s 19ms/step - loss: 0.2721 - accuracy: 0.8959 - val_loss: 0.3340 - val_accuracy: 0.8789
Epoch 3/200
169/169 [==============================] - 3s 19ms/step - loss: 0.2244 - accuracy: 0.9153 - val_loss: 0.2324 - val_accuracy: 0.9117
Epoch 4/200
169/169 [==============================] - 3s 19ms/step - loss: 0.1736 - accuracy: 0.9359 - val_loss: 0.2153 - val_accuracy: 0.9289
Epoch 5/200
169/169 [==============================] - 3s 20ms/step - loss: 0.1559 - accuracy: 0.9424 - val_loss: 0.2467 - val_accuracy: 0.9228
Epoch 6/200
169/169 [==============================] - 3s 20ms/step - loss: 0.1487 - accuracy: 0.9377 - val_loss: 0.1869 - val_accuracy: 0.9317
Epoch 7/200
169/169 [==============================] - 3s 21ms/step - loss: 0.1359 - accuracy: 0.9468 - val_loss: 0.1565 - val_accurac

In [21]:
model.evaluate(conv_base.predict(X_test),y_test)

57/57 [==============================] - 0s 2ms/step - loss: 0.1446 - accuracy: 0.9689


[0.14458876848220825, 0.9688888788223267]

In [22]:
save_model(model,'./self_model/20240118_categorical_model.h5')
save_model(conv_base,'./self_model/20240118_VGG16.h5')

C:\Users\user\AppData\Local\Temp\ipykernel_21056\410920196.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model,'./self_model/20240118_categorical_model.h5')
C:\Users\user\AppData\Local\Temp\ipykernel_21056\410920196.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(conv_base,'./self_model/20240118_VGG16.h5')


In [23]:
model=load_model('./self_model/20240118_categorical_model.h5')
conv_base=load_model('./self_model/20240118_VGG16.h5')

In [24]:
def model_predict(x):
    x1 = cv.resize(x,(150,150))
    x1 = x1[:,:,:3]
    x1 = x1.reshape(1,150,150,3)
    
    x1_trans=conv_base.predict(x1)
    
    answer=np.argmax(model.predict(x1_trans))
    return answer

In [25]:
###같은 사진의 밝기 변화에 의한 예측값 변화 확인 테스트
X_raw=plt.imread('./self_image/bright/raw.jpg')
X_bright40=plt.imread('./self_image/bright/bright40.jpg')
X_bright60=plt.imread('./self_image/bright/bright60.jpg')
X_bright80=plt.imread('./self_image/bright/bright80.jpg')
X_bright100=plt.imread('./self_image/bright/bright100.jpg')
X_dark30=plt.imread('./self_image/bright/dark30.jpg')
X_dark60=plt.imread('./self_image/bright/dark60.jpg')
X_dark90=plt.imread('./self_image/bright/dark90.jpg')
X_gray=plt.imread('./self_image/bright/gray.jpg')
X_tintminus100=plt.imread('./self_image/bright/tintminus100.jpg')
X_tintplus100=plt.imread('./self_image/bright/tintplus100.jpg')

In [26]:
print(model_predict(X_raw))
print(model_predict(X_bright40))
print(model_predict(X_bright60))
print(model_predict(X_bright80))
print(model_predict(X_bright100))
print(model_predict(X_dark30))
print(model_predict(X_dark60))
print(model_predict(X_dark90))
print(model_predict(X_gray))
print(model_predict(X_tintminus100))
print(model_predict(X_tintplus100))

1/1 [==============================] - 0s 64ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 20ms/step
1
1/1 [==============================] - 0s 20ms/step
1
1/1 [==============================] - 0s 29ms/step
1
1/1 [==============================] - 0s 20ms/step
1
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 20ms/step
1


In [27]:
X_20240119_10=plt.imread('./self_image/self/20240119_10.jpg')
X_20240119_11=plt.imread('./self_image/self/20240119_11.jpg')
X_20240119_12=plt.imread('./self_image/self/20240119_12.jpg')
X_20240119_13=plt.imread('./self_image/self/20240119_13.jpg')
X_20240119_14=plt.imread('./self_image/self/20240119_14.jpg')
X_20240119_15=plt.imread('./self_image/self/20240119_15.jpg')
X_20240119_16=plt.imread('./self_image/self/20240119_16.jpg')

In [28]:
print(model_predict(X_20240119_10))
print(model_predict(X_20240119_11))
print(model_predict(X_20240119_12))
print(model_predict(X_20240119_13))
print(model_predict(X_20240119_14))
print(model_predict(X_20240119_15))
print(model_predict(X_20240119_16))

1/1 [==============================] - 0s 20ms/step
6
1/1 [==============================] - 0s 23ms/step
6
1/1 [==============================] - 0s 25ms/step
6
1/1 [==============================] - 0s 24ms/step
6
1/1 [==============================] - 0s 22ms/step
6
1/1 [==============================] - 0s 25ms/step
6
1/1 [==============================] - 0s 21ms/step
6


In [29]:
# labels_dic = {
# "0to1000" : 0
# ,"1000to2000" : 1
# ,"2000to3000" : 2
# ,"2000to4000" : 3
# ,"3000to5000" : 4
# ,"4000to6000" : 5
# ,"5000to7000" : 6
# }



# labels_dic = {v:k for k,v in labels_dic.items()} 

In [30]:
# x=plt.imread('./self_image/20240119_12.jpg')
# x = cv.resize(x,(150,150))
# x = x[:,:,:3]
# x = x.reshape(1,150,150,3)

# conv_base=VGG16(include_top=False, input_shape=(150,150,3))
# x_trans=conv_base.predict(x)

# model_20240119.predict(x_trans)
# np.argmax(model_20240119.predict(x_trans))
# #answer=labels_dic.get(np.argmax(model_20240119.predict(x_trans)))



In [31]:
# def model_predict(x):
#     x1 = cv.resize(x,(150,150))
#     x1 = x1[:,:,:3]
#     x1 = x1.reshape(1,150,150,3)
    
#     conv_base=VGG16(include_top=False, input_shape=(150,150,3))
#     x1_trans=conv_base.predict(x1)
    
#     answer=np.argmax(model_20240119.predict(x_trans))
#     return answer

In [32]:
# x=plt.imread('./self_image/self/20240119_12.jpg')
# model_predict(x)